In [49]:
# импортруем библиотеку pandas для считывания файла и дальнейшей обработки таблицы
import pandas as pd

data = pd.read_csv("data.tsv", sep='\t', header=None, names=['search', 'date', 'device'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1114365 entries, 0 to 1114364
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   search  1114365 non-null  object
 1   date    1114365 non-null  int64 
 2   device  1114365 non-null  object
dtypes: int64(1), object(2)
memory usage: 25.5+ MB


In [50]:
# переводим время в UTC
UTC_date = pd.to_datetime(data['date'], unit='s')

data['date'] = UTC_date

data.head() #сомтрим на структуру таблицы

,search,date,device
0,малевич картины,2021-09-16 15:34:25,desktop
1,психология,2021-09-16 08:39:43,touch
2,с днём рождения лена,2021-09-16 05:52:43,touch
3,зверополис фильмы,2021-09-16 10:19:59,touch
4,алабай собака фото,2021-09-16 10:04:05,touch


Находим временной диапазон данных

In [51]:
data['date'].describe()
print('Временной диапазон данных: c ', data['date'].min(), 'по', data['date'].max())

Временной диапазон данных: c  2021-08-31 21:00:00 по 2021-09-21 20:59:59


Определяем количество запросов со словом 'ютуб' с разных девайсов.
Для написания запросв внутри кода используем библиотеку sqldf

In [52]:
from pandasql import sqldf

query2 = """
SELECT device, COUNT(search) as Количество
FROM data
WHERE LOWER(search) like '%ютуб%'
GROUP BY device
ORDER BY Количество DESC
"""
print('Кол-во запросов со словом ютуб:')
result2 = sqldf(query2, globals())
print(result2)

Кол-во запросов со словом ютуб:
    device  Количество
0  desktop         802
1    touch         597


Определяем топ 10 самых популярных запросов по каждому девайсу

In [53]:
# Топ‑10 самых частых запрсов по каждому 'device' sql запрос
query3 = """
SELECT device, search, count
FROM (
    SELECT
        device,
        search,
        COUNT(*) AS count,
        ROW_NUMBER() OVER (PARTITION BY device ORDER BY COUNT(*) DESC) AS rn
    FROM data
    GROUP BY device, search
)
WHERE rn <= 10
ORDER BY device, rn
"""

result3 = sqldf(query3, globals())
print(result3)

     device                           search  count
0   desktop                   календарь 2021   2804
1   desktop               таблица менделеева   2631
2   desktop                         картинки   1647
3   desktop               английский алфавит   1293
4   desktop             обои на рабочий стол   1143
5   desktop  Одноклассники (социальная сеть)   1116
6   desktop                таблица квадратов    877
7   desktop                          алфавит    874
8   desktop                таблица умножения    867
9   desktop                       карта мира    795
10    touch          с днём рождения женщине   4903
11    touch                  с днём рождения   3967
12    touch          с днём рождения мужчине   3623
13    touch                  с днем рождения   3005
14    touch                           погода   2840
15    touch                             игры   2833
16    touch                           фильмы   2746
17    touch                          новости   2708
18    touch 

Тот же самый запрос, но при использовании библиотека pandas

In [61]:
# Топ‑10 самых частых запрсов по каждому 'device' при использовании библиотеки pandas
result3 = data.groupby('device')['search'].value_counts()
print('Топ‑10 самых частых запрсов по каждому девайсу')
print(result3.groupby('device').head(10))

Топ‑10 самых частых запрсов по каждому девайсу
device   search                         
desktop  календарь 2021                     2804
         таблица менделеева                 2631
         картинки                           1647
         английский алфавит                 1293
         обои на рабочий стол               1143
         Одноклассники (социальная сеть)    1116
         таблица квадратов                   877
         алфавит                             874
         таблица умножения                   867
         карта мира                          795
touch    с днём рождения женщине            4903
         с днём рождения                    3967
         с днём рождения мужчине            3623
         с днем рождения                    3005
         погода                             2840
         игры                               2833
         фильмы                             2746
         новости                            2708
         музыка               

Можно заметить различия в запросах с разных устройств. С ПК пользователи чаще всего ищут информацию, связанную с обучением. С мобильных устройств в поиске можно часто встретить ежедневные рутинные запросы - про погоду, новости, фильмы, но самыми частыми запросами являются поиск текста или картинки поздравления.

Определяем траффик запросов в течекние дня

In [24]:
query4 = """
SELECT strftime('%H', date) as day, device, COUNT(search) as КоличествоЗапросов
FROM data
GROUP BY strftime('%H', date), device
ORDER BY COUNT(search) DESC
"""
result4 = sqldf(query4, globals())
print(result4)

   day   device  КоличествоЗапросов
0   17    touch               46080
1   18    touch               43867
2   07    touch               43224
3   06    touch               43138
4   16    touch               40641
5   05    touch               40620
6   15    touch               39496
7   14    touch               38510
8   13    touch               38022
9   12    touch               37216
10  09    touch               37206
11  08    touch               37157
12  11    touch               37108
13  10    touch               36577
14  19    touch               36430
15  04    touch               29063
16  13  desktop               27013
17  12  desktop               26354
18  14  desktop               25864
19  16  desktop               25551
20  11  desktop               25475
21  15  desktop               25349
22  17  desktop               24834
23  10  desktop               24227
24  09  desktop               23299
25  20    touch               22930
26  08  desktop             

Опираясь на полученные данные, можно сделать несколько выводов. Во-первых, чаще всего пользователи используют мобильные устройства для поиска. Во-вторых, заметно, что использование активное компьютера приходится на дневные часы - с 9 до 17, можно предположить, что ПК используют для работы или во время учебы. С мобильного телефона много запросов в течение всего дня, но особенно выделяются часы под конец рабочего дня - 16-18 и начало - 5-7, так как мобильный телефон чаще всего находится всегдня рядом.

Для понимания на какие тематики нужно разбить запросы, выделяем самые встречающиеся запросы

In [35]:
query31 = """
SELECT device, search, count
FROM (
    SELECT
        device,
        search,
        COUNT(*) AS count
    FROM data
    GROUP BY device, search
)
ORDER BY count
"""

result31 = sqldf(query31, globals())

Записываем полученные данные в файл

In [47]:
data.to_csv('data.csv', index = False)
result4.to_csv('daytraffic.csv', index = False)
result3.to_csv('topquery.csv', index = False)
result31.to_csv('topqueryAll.csv', index = False)

In [8]:
# Создаём копию датасета, чтобы сохранить оригинал данных
result5 = data.copy()
# Проверяем уникальные значения запросов
result5['search'].unique()

In [57]:
# Создаем колнку категории
result5['category'] = ''

# Функция по распределению запросов по категориям
def assign_category(text):
    if pd.isna(text):
        return 'Нет данных'
    text_lower = text.lower()
    
    if any(word in text_lower for word in ['днём', 'день', 'праздн', 'поздрав', 'юбилей', 'годовщин', 'свадьб', 'рождество', 'новый год']):
        return 'Поздравления'
    elif any(word in text_lower for word in ['алфавит', 'логия', 'таблица', 'карта', 'закон', 'теорем', 'аксиом', 'формул', 'учебник', 'гдз']):
        return 'Учёба'
    elif any(word in text_lower for word in ['фильм', 'мульт', 'видео', 'мем', 'музык', 'подкаст', 'игр', 'кино', 'смотреть', 'слушать']):
        return 'Развлечения'
    elif any(word in text_lower for word in ['билет', 'тур', 'концерт', 'театр', 'кино']):
        return 'Покупка билетов'
    elif any(word in text_lower for word in ['погода']):
        return 'Погода'
    elif any(word in text_lower for word in ['новост', 'спорт', 'наука', 'технологи']):
        return 'Новости'
    elif any(word in text_lower for word in ['дизайн', 'интерьер', 'лофт', 'декор']):
        return 'Дизайн'
    elif any(word in text_lower for word in ['картина', 'картины', 'набросок', 'гравюра', 'произведени']):
        return 'Искусство'
    elif any(word in text_lower for word in ['картинк', 'фото', 'обои']):
        return 'Картинки'
    elif any(word in text_lower for word in ['вк', 'вконтакт', 'ютуб', 'тг', 'одноклассник']):
        return 'СоцСети'
    elif any(word in text_lower for word in ['порн', 'прон', 'ххх', 'женщин', 'эротик']):
        return '18+'
    else:
        return 'Другое'

# Применяем функцию ко всем запросам, заполняем столбец Категория
result5['category'] = result5['search'].apply(assign_category)
# Смотрим результат
result5.head(10)


,search,date,device,category
0,малевич картины,2021-09-16 15:34:25,desktop,Искусство
1,психология,2021-09-16 08:39:43,touch,Учёба
2,с днём рождения лена,2021-09-16 05:52:43,touch,Поздравления
3,зверополис фильмы,2021-09-16 10:19:59,touch,Развлечения
4,алабай собака фото,2021-09-16 10:04:05,touch,Картинки
5,бактериофаг,2021-09-16 18:16:42,desktop,Другое
6,полина гагарина,2021-09-16 14:53:25,touch,Другое
7,кадр из фильма,2021-09-16 05:40:37,desktop,Развлечения
8,погода,2021-09-16 08:25:41,touch,Погода
9,кухни дизайн,2021-09-16 17:37:01,touch,Дизайн


In [56]:
result5.to_csv('category.csv', index = False, encoding='utf-8')